In [ ]:
from d123.dataset.maps.gpkg.gpkg_map import get_map_api_from_names

map_api = get_map_api_from_names("carla", "town05")

In [ ]:
from d123.common.visualization.color.default import MAP_SURFACE_CONFIG
from d123.dataset.maps.map_datatypes import MapSurfaceType

from typing import List
from shapely import LineString, points
import shapely
from d123.dataset.dataset_specific.nuplan.nuplan_map_conversion import get_road_edge_linestrings

import matplotlib.pyplot as plt

from d123.common.visualization.color.color import BLACK, TAB_10
from d123.common.visualization.color.config import PlotConfig
from d123.common.visualization.matplotlib.utils import add_shapely_linestring_to_ax, add_shapely_polygon_to_ax

lane_groups_ = map_api._gpd_dataframes[MapSurfaceType.LANE_GROUP].geometry.tolist()
car_parks_ = map_api._gpd_dataframes[MapSurfaceType.CARPARK].geometry.tolist()
generic_drivable_ = map_api._gpd_dataframes[MapSurfaceType.GENERIC_DRIVABLE].geometry.tolist()


s = 30
fig, ax = plt.subplots(figsize=(s, s))
for polygon in lane_groups_:
    add_shapely_polygon_to_ax(ax, polygon, MAP_SURFACE_CONFIG[MapSurfaceType.LANE_GROUP])
for polygon in car_parks_:
    add_shapely_polygon_to_ax(ax, polygon, MAP_SURFACE_CONFIG[MapSurfaceType.CARPARK])
for polygon in generic_drivable_:
    add_shapely_polygon_to_ax(ax, polygon, MAP_SURFACE_CONFIG[MapSurfaceType.WALKWAY])

In [ ]:
import numpy as np
from shapely import union_all
from d123.common.geometry.occupancy_map import OccupancyMap2D
from d123.dataset.maps.abstract_map_objects import AbstractLane
from d123.dataset.maps.map_datatypes import MapSurfaceType

all_lane_group_ids = map_api._gpd_dataframes[MapSurfaceType.LANE_GROUP].id.tolist()

all_lane_groups = [map_api.get_map_object(id, MapSurfaceType.LANE_GROUP) for id in all_lane_group_ids]
# lane_groups_no_intersection = [lane_group for lane_group in all_lane_groups if lane_group.intersection is None]
lane_groups_no_intersection = all_lane_groups

occupancy_map = OccupancyMap2D.from_dict(
    {lane_group.id: lane_group.shapely_polygon for lane_group in all_lane_groups}
)


def get_z_at_point(lane_group: AbstractLane, point: np.ndarray) -> float:
    """
    Placeholder function to get the z-coordinate at a point.
    This should be replaced with the actual implementation that retrieves the z-coordinate.
    """
    centerlines = np.concatenate([lane.centerline.array for lane in lane_group.lanes], axis=0, dtype=np.float64)
    distances_2d = np.linalg.norm(centerlines[..., :2] - point[:2], axis=-1)
    closest_lane_index = np.argmin(distances_2d)
    return centerlines[closest_lane_index, 2]


overlapping_lane_groups = {}

for lane_group in lane_groups_no_intersection:
    lane_centerlines = np.array([lane.centerline.array for lane in lane_group.lanes], dtype=np.float64)

    intersecting_lane_groups = occupancy_map.intersects(lane_group.shapely_polygon)
    intersecting_lane_groups.remove(lane_group.id)
    lane_group_polygon = lane_group.shapely_polygon

    for intersecting_id in intersecting_lane_groups:
        intersecting_polygon = occupancy_map[intersecting_id]
        intersection = intersecting_polygon.intersection(lane_group_polygon)
        if intersection.geom_type != "Polygon":
            continue

        intersecting_lane_group = map_api.get_map_object(intersecting_id, MapSurfaceType.LANE_GROUP)
        z_at_query = get_z_at_point(lane_group, intersection.centroid.coords[0])
        z_at_inter = get_z_at_point(intersecting_lane_group, intersection.centroid.coords[0])

        if np.abs(z_at_query - z_at_inter) < 5.0:
            continue

        overlapping_lane_groups[lane_group.id] = lane_group
        overlapping_lane_groups[intersecting_lane_group.id] = intersecting_lane_group

# occupancy_map

In [ ]:
non_overlapping_lane_groups = [
    lane_group for lane_group in lane_groups_no_intersection if lane_group.id not in overlapping_lane_groups
]


def get_linestring_z_at_point(linestring: LineString, point: np.ndarray) -> float:
    assert linestring.has_z, "Linestring must have z-coordinates"
    linestring_coords = np.array(list(linestring.coords), dtype=np.float64)
    distances_2d = np.linalg.norm(linestring_coords[:, :2] - point[:2], axis=-1)
    closest_point_index = np.argmin(distances_2d)
    return float(linestring_coords[closest_point_index, 2])


def lift_road_edges_to_3d(ax: plt.Axes, road_edges_2d: List[LineString]):
    def _find_continuous_sublists(integers: List[int]) -> List[List[int]]:
        arr = np.array(integers, dtype=np.int64)
        breaks = np.where(np.diff(arr) != 1)[0] + 1
        splits = np.split(arr, breaks)
        return [sublist.tolist() for sublist in splits]

    boundaries = []
    for lane_group in non_overlapping_lane_groups:
        boundaries.append(lane_group.left_boundary.linestring)
        boundaries.append(lane_group.right_boundary.linestring)

    boundaries.extend(
        map_api._gpd_dataframes[MapSurfaceType.CARPARK].outline.tolist()
        + map_api._gpd_dataframes[MapSurfaceType.GENERIC_DRIVABLE].outline.tolist()
    )

    occupancy_map = OccupancyMap2D(boundaries)

    road_edges_3d: List[LineString] = []
    for idx, linestring in enumerate(road_edges_2d):
        # print(list(linestring.coords))
        points_3d = np.array(list(linestring.coords), dtype=np.float64)

        results = occupancy_map.query_nearest(
            shapely.points(points_3d[..., :2]),
            max_distance=0.01,
            exclusive=False,
        )
        for query_idx, geometry_idx in zip(*results):
            intersecting_boundary = occupancy_map[occupancy_map.ids[geometry_idx]]
            points_3d[query_idx, 2] = get_linestring_z_at_point(intersecting_boundary, points_3d[query_idx])

        for continuous_slice in _find_continuous_sublists(results[0]):
            if len(continuous_slice) < 2:
                continue
            lifted_linestring = LineString(points_3d[continuous_slice])
            road_edges_3d.append(lifted_linestring)
    return road_edges_3d


drivable_polygons = (
    map_api._gpd_dataframes[MapSurfaceType.LANE_GROUP].geometry.tolist()
    + map_api._gpd_dataframes[MapSurfaceType.CARPARK].geometry.tolist()
    + map_api._gpd_dataframes[MapSurfaceType.GENERIC_DRIVABLE].geometry.tolist()
)

road_edges_2d = get_road_edge_linestrings(drivable_polygons, step_size=0.25, max_road_edge_length=None)


s = 30
fig, ax = plt.subplots(figsize=(s, s))
road_edges_3d = lift_road_edges_to_3d(ax, road_edges_2d)



for lane_group in overlapping_lane_groups.values():
    road_edges_3d.extend([lane_group.right_boundary.linestring, lane_group.left_boundary.linestring])


for idx, linestring in enumerate(road_edges_3d):
    config = PlotConfig(
        fill_color=TAB_10[idx % len(TAB_10)], fill_color_alpha=0.25, line_color=TAB_10[idx % len(TAB_10)]
    )
    # config = PlotConfig(fill_color=BLACK, fill_color_alpha=0.25, line_color=BLACK)
    add_shapely_linestring_to_ax(ax, linestring, config)

In [ ]:
def find_continuous_sublists(arr):
    arr = np.array(arr)
    breaks = np.where(np.diff(arr) != 1)[0] + 1
    splits = np.split(arr, breaks)
    return [sublist.tolist() for sublist in splits]

# Example usage
arr = [0, 1, 2, 3, 5, 6, 7, 9, 10, 22, 23, 24]
result = find_continuous_sublists(arr)
print(result)  # [[0, 1, 2, 3], [5, 6, 7]]

In [ ]:

s = 30

fig, ax = plt.subplots(figsize=(s, s))


for idx, lane_group in enumerate(overlapping_lane_groups.values()):
    config = PlotConfig(fill_color=TAB_10[idx % len(TAB_10)], fill_color_alpha=0.25, line_color=TAB_10[idx % len(TAB_10)])
    add_shapely_polygon_to_ax(ax, lane_group.shapely_polygon, config)

# for idx, polygon in enumerate(intersections):
#     config = PlotConfig(fill_color=BLACK, line_color=BLACK)
#     add_shapely_polygon_to_ax(ax, polygon, config)

ax.set_aspect("equal")